In [121]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [122]:
language="it"
modelname="ItalianBert"
input = "gpt4"
test = "Dodiom"
filtered= True

In [123]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [124]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'train_gpt4_it_zero_shot.csv')
df_eval = pd.read_csv(f'eval_gpt4_it_zero_shot.csv')
df_test = pd.read_csv(f'dodiom_it_test.csv')

In [125]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [126]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'figurato': 'idiom', 'letterale': 'nonidiom'}


In [127]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,submission,category,idiom,type
0,In vista della tempesta economica che si avvic...,idiom,Ammainare le vele,Zero-shot
1,La trattativa sul contratto sembra essersi inc...,idiom,Essere su un binario morto,Zero-shot
2,"Nel picco del traffico, ogni semaforo rosso se...",idiom,essere tra i piedi,Zero-shot
3,"Ogni volta che provo a dedicarmi alla lettura,...",idiom,essere tra i piedi,Zero-shot
4,"Nelle grandi aziende, a volte è preferibile co...",idiom,Coltivare il proprio orto,Zero-shot
...,...,...,...,...
3547,"Presto, metti la torta dentro il forno per cuo...",nonidiom,Mettere dentro,Zero-shot
3548,"Durante la competizione, ogni squadra doveva r...",nonidiom,Raggiungere il bersaglio,Zero-shot
3549,"Al rientro dal sentiero, tutti dovettero esser...",nonidiom,Essere a cavallo,Zero-shot
3550,L'attore dovette voltare rapidamente la propri...,nonidiom,Voltare la faccia,Zero-shot


In [128]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,submission,category,idiom,type
0,Questo film non ha rispettato le mie aspettati...,idiom,Rifarsi la bocca,Zero-shot
1,"Dopo una giornata frenetica, arrivo a casa con...",idiom,Avere le batterie scariche,Zero-shot
2,Avere un capo così esigente è come andare in u...,idiom,Aver la corda al collo,Zero-shot
3,Voltare la faccia alla corruzione equivale a s...,idiom,Voltare la faccia,Zero-shot
4,"Con il nuovo aumento delle tasse, molte famigl...",idiom,Aver la corda al collo,Zero-shot
...,...,...,...,...
883,Tentando di mantenere la concentrazione al tir...,nonidiom,Avere ancora della birra nel corpo,Zero-shot
884,Ingoiare una pillola può essere complicato se ...,nonidiom,Ingoiare una pillola,Zero-shot
885,"Dopo aver raccolto le patate, tiriamo su con c...",nonidiom,Tirare su,Zero-shot
886,Il ristorante ha deciso di mandare gli avanzi ...,nonidiom,mandare a casa,Zero-shot


In [129]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [130]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '..."
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'..."
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare..."
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior..."
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare..."
...,...,...,...,...,...,...,...,...,...,...,...
2279,voltare la faccia,"Puoi voltare la faccia per favore, voglio fart...",nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['potere', 'voltare', 'la', 'faccia', 'per', '...","['Puoi', 'voltare', 'la', 'faccia', 'per', 'fa..."
2280,voltare la faccia,per favore puoi voltare la faccia verso di me ...,nonidiom,1.000000,3,0,0,"[3, 5]","['voltare', 'faccia']","['per', 'favore', 'potere', 'voltare', 'la', '...","['per', 'favore', 'puoi', 'voltare', 'la', 'fa..."
2281,voltare la faccia,devi voltare la faccia da entrambi i lati per ...,nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'da', '...","['devi', 'voltare', 'la', 'faccia', 'da', 'ent..."
2282,voltare la faccia,Appena mi ha visto ha voltato la faccia,nonidiom,0.500000,1,1,0,"[5, 7]","['voltato', 'faccia']","['appena', 'mi', 'avere', 'vistare', 'avere', ...","['Appena', 'mi', 'ha', 'visto', 'ha', 'voltato..."


In [131]:
if filtered == True:
  df_train = df_train.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=32, replace=False)).reset_index(drop=True)
  df_eval = df_eval.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=8, replace=False)).reset_index(drop=True)

<ipython-input-131-5d2163afaadf>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = df_train.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=32, replace=False)).reset_index(drop=True)
<ipython-input-131-5d2163afaadf>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_eval = df_eval.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=8, replace=False)).reset_index(drop=True)


In [132]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [133]:
df_train.head()

,submission,category,idiom,type
0,"Per quanto creativi, i suoi progetti sembrano ...",idiom,Acchiappare farfalle,Zero-shot
1,"In questa fase della sua vita, sembra più inte...",idiom,Acchiappare farfalle,Zero-shot
2,Acchiappare farfalle nel mondo degli affari pu...,idiom,Acchiappare farfalle,Zero-shot
3,Quando si tratta di fare piani a lungo termine...,idiom,Acchiappare farfalle,Zero-shot
4,"Spesso, nei momenti di decisione, si perde ad ...",idiom,Acchiappare farfalle,Zero-shot


In [134]:
df_eval.head()

,submission,category,idiom,type
0,Criticare senza agire è come acchiappare farfa...,idiom,Acchiappare farfalle,Zero-shot
1,"Mentre tutti erano concentrati sul compito, lu...",idiom,Acchiappare farfalle,Zero-shot
2,Acchiappare farfalle può essere un modo per ev...,idiom,Acchiappare farfalle,Zero-shot
3,"Nel suo nuovo romanzo, l'autore ha il vizio di...",idiom,Acchiappare farfalle,Zero-shot
4,"Durante la riunione, sembrava che stesse acchi...",idiom,Acchiappare farfalle,Zero-shot


In [135]:
df_test.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '..."
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'..."
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare..."
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior..."
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare..."


In [136]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [137]:
df_train

,submission,category,idiom,type,category_encoded
0,"Per quanto creativi, i suoi progetti sembrano ...",idiom,Acchiappare farfalle,Zero-shot,0
1,"In questa fase della sua vita, sembra più inte...",idiom,Acchiappare farfalle,Zero-shot,0
2,Acchiappare farfalle nel mondo degli affari pu...,idiom,Acchiappare farfalle,Zero-shot,0
3,Quando si tratta di fare piani a lungo termine...,idiom,Acchiappare farfalle,Zero-shot,0
4,"Spesso, nei momenti di decisione, si perde ad ...",idiom,Acchiappare farfalle,Zero-shot,0
...,...,...,...,...,...
2363,"Durante la pausa estiva, la scuola mandava i l...",nonidiom,mandare a casa,Zero-shot,1
2364,"Per comodità dei partecipanti, abbiamo mandato...",nonidiom,mandare a casa,Zero-shot,1
2365,"Per assistere gli anziani, il comune manda i p...",nonidiom,mandare a casa,Zero-shot,1
2366,L'officina manda a casa i veicoli riparati dop...,nonidiom,mandare a casa,Zero-shot,1


In [138]:
df_eval

,submission,category,idiom,type,category_encoded
0,Criticare senza agire è come acchiappare farfa...,idiom,Acchiappare farfalle,Zero-shot,0
1,"Mentre tutti erano concentrati sul compito, lu...",idiom,Acchiappare farfalle,Zero-shot,0
2,Acchiappare farfalle può essere un modo per ev...,idiom,Acchiappare farfalle,Zero-shot,0
3,"Nel suo nuovo romanzo, l'autore ha il vizio di...",idiom,Acchiappare farfalle,Zero-shot,0
4,"Durante la riunione, sembrava che stesse acchi...",idiom,Acchiappare farfalle,Zero-shot,0
...,...,...,...,...,...
587,La profumeria manda campioni gratuiti a casa d...,nonidiom,mandare a casa,Zero-shot,1
588,L'impresa edile manda a casa dei lavoratori tu...,nonidiom,mandare a casa,Zero-shot,1
589,L'agenzia di viaggi manda i documenti necessar...,nonidiom,mandare a casa,Zero-shot,1
590,Manda queste piante a casa in modo che possano...,nonidiom,mandare a casa,Zero-shot,1


In [139]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,abbaiare alla luna,Abbaiare alla luna o roteare gatti non può cur...,idiom,1.000000,2,0,0,"[0, 1, 2]","['Abbaiare', 'alla', 'luna']","['abbaiare', 'alla', 'luna', 'o', 'roteare', '...","['Abbaiare', 'alla', 'luna', 'o', 'roteare', '...",0
1,abbaiare alla luna,Non è possibile che tu debba abbaiare per tutt...,idiom,0.500000,1,1,0,"[6, 9, 10]","['abbaiare', 'alla', 'luna']","['non', 'essere', 'possibile', 'che', 'tu', 'd...","['Non', 'è', 'possibile', 'che', 'tu', 'debba'...",0
2,abbaiare alla luna,Chiedere alla prof di pubblicare le slide è co...,idiom,0.500000,1,1,0,"[9, 10, 11]","['abbaiare', 'alla', 'luna']","['chiedere', 'alla', 'prof', 'di', 'pubblicare...","['Chiedere', 'alla', 'prof', 'di', 'pubblicare...",0
3,abbaiare alla luna,Abbai almeno 10 volte al giorno alla luna,idiom,0.000000,0,2,0,"[0, 6, 7]","['Abbai', 'alla', 'luna']","['abbaiare', 'almeno', '10', 'volta', 'al', 'g...","['Abbai', 'almeno', '10', 'volte', 'al', 'gior...",0
4,abbaiare alla luna,Sprechi troppo tempo ad abbaiare alla luna,idiom,0.666667,2,1,0,"[4, 5, 6]","['abbaiare', 'alla', 'luna']","['sprecare', 'troppo', 'tempo', 'ad', 'abbaiar...","['Sprechi', 'troppo', 'tempo', 'ad', 'abbaiare...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
2279,voltare la faccia,"Puoi voltare la faccia per favore, voglio fart...",nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['potere', 'voltare', 'la', 'faccia', 'per', '...","['Puoi', 'voltare', 'la', 'faccia', 'per', 'fa...",1
2280,voltare la faccia,per favore puoi voltare la faccia verso di me ...,nonidiom,1.000000,3,0,0,"[3, 5]","['voltare', 'faccia']","['per', 'favore', 'potere', 'voltare', 'la', '...","['per', 'favore', 'puoi', 'voltare', 'la', 'fa...",1
2281,voltare la faccia,devi voltare la faccia da entrambi i lati per ...,nonidiom,1.000000,3,0,0,"[1, 3]","['voltare', 'faccia']","['deviare', 'voltare', 'la', 'faccia', 'da', '...","['devi', 'voltare', 'la', 'faccia', 'da', 'ent...",1
2282,voltare la faccia,Appena mi ha visto ha voltato la faccia,nonidiom,0.500000,1,1,0,"[5, 7]","['voltato', 'faccia']","['appena', 'mi', 'avere', 'vistare', 'avere', ...","['Appena', 'mi', 'ha', 'visto', 'ha', 'voltato...",1


In [140]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [141]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [142]:
num_labels = len(label_encoder.classes_)
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-italian-cased", num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/296 [00:00<?, ?it/s]<ipython-input-140-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-140-845b27163e62>:8: U

Epoch 1/4 | Train Loss: 0.5079682629706489


100%|██████████| 74/74 [00:01<00:00, 52.68it/s]


Epoch 1/4 | Validation Accuracy: 95.61%


100%|██████████| 296/296 [00:23<00:00, 12.56it/s]


...

Epoch 3/4 | Validation Accuracy: 99.16%


100%|██████████| 296/296 [00:27<00:00, 10.87it/s]


Epoch 4/4 | Train Loss: 0.030900186494093488


100%|██████████| 74/74 [00:01<00:00, 53.81it/s]

Epoch 4/4 | Validation Accuracy: 98.65%


In [143]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


<ipython-input-140-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-140-845b27163e62>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Test Accuracy: 0.6799
Weighted F1 Score: 0.6740
Macro F1 Score: 0.6782


In [144]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)

Classification Report for ItalianBert, language IT, Train GPT4, Test DODIOM:
              precision    recall  f1-score   support

           0     0.8977    0.5152    0.6547      1345
           1     0.5688    0.9159    0.7018       939

    accuracy                         0.6799      2284
   macro avg     0.7332    0.7156    0.6782      2284
weighted avg     0.7625    0.6799    0.6740      2284

